In [ ]:
from pathlib import Path
from functools import partial

import torch
from torch.nn.functional import softmax
from torch.utils.data import random_split

import matplotlib.pyplot as plt
import seaborn as sns

from ballchallenge.model import BallChallengeModel
from ballchallenge.accelerometer_dataset import AccelerometerDataset
from ballchallenge.training import run_training

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DATASET_ROOT = Path("../data")
GRID_SIZE = (10, 10)

In [ ]:
def flat_labels(labels: torch.Tensor) -> torch.Tensor:
    return labels.flatten(start_dim=1)

def downsample(samples: torch.Tensor, factor: int) -> torch.Tensor:
    return samples[:,:,::factor]

ds = AccelerometerDataset(
    dataset_root=DATASET_ROOT,
    grid_size=GRID_SIZE,
    x_position_range=(0, 2),
    y_position_range=(0, 2),
    label_std=0.3,
    transform_samples=partial(downsample, factor=4),
    transform_labels=flat_labels,
)
ds_train, ds_test = random_split(ds, lengths=[0.75, 0.25])

print("Train Samples:", ds_train[:][0].shape)
print("Test Samples:", ds_test[:][0].shape)

In [ ]:
model = BallChallengeModel(total_bits=16, frac_bits=8, signal_length=ds_train[0][0].shape[1], grid_size=GRID_SIZE)

sum(param.numel() for param in model.parameters())

In [ ]:
history = run_training(
    model=model,
    ds_train=ds_train,
    ds_test=ds_test,
    batch_size=8,
    epochs=500,
    learning_rate=1e-3,
    device=DEVICE
)

model.eval()
model.to("cpu")

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axs[0].plot(history.train["epoch"], history.train["loss"], label="train")
axs[0].plot(history.test["epoch"], history.test["loss"], label="test")
axs[0].set_xlabel("Epoch")
axs[0].set_ylabel("Loss")
axs[0].legend()

axs[1].plot(history.train["epoch"], history.train["accuracy"], label="train")
axs[1].plot(history.test["epoch"], history.test["accuracy"], label="test")
axs[1].set_xlabel("Epochs")
axs[1].set_ylabel("Accuracy")
axs[1].legend()

In [ ]:
sample_idx = 0

sample, target = ds_test[sample_idx]
prediction = softmax(model(sample).detach(), dim=0).view(*GRID_SIZE[::-1])
target = target.view(*GRID_SIZE[::-1])

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

sns.heatmap(prediction, cmap="hot", square=True, ax=axs[0])
axs[0].set_title("Prediction")

sns.heatmap(target, cmap="hot", square=True, ax=axs[1])
axs[1].set_title("Target")

In [ ]:
_, labels = ds[:]
mean_label = labels.mean(dim=0)
sns.heatmap(mean_label.view(GRID_SIZE), square=True, cmap="hot")